# Tree-based methods and ensembles

@author: Jan Verwaeren - Arne Deloose - Nusret Ipek

@course: Machine learning: van theorie tot praktijk

This notbook contains illustrations of the most prominent tree-based methods and ensemble methods used in the ML field:
- CART: classification and regression trees (Breiman 1984)
- Random forests
- Gradient Boosting (XGBoost)
- Auto-ml
- Isolation forests


## Import libraries

******


In [1]:
# Basic libraries
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", RuntimeWarning)
warnings.simplefilter("ignore", DeprecationWarning)

# Sklearn
## Data
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

## Models
from sklearn import tree
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.manifold import TSNE

## Model Explaination
from sklearn.inspection import permutation_importance
from sklearn.inspection import PartialDependenceDisplay

## Metrics
from sklearn.metrics import accuracy_score

# XGBoost
import xgboost

# Plotting
import graphviz
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display



## 1. The Wisonsin breast cancer dataset

******

In this demo notebook, the Wisconsin breast cancer dataset is used. Several features derived from microscopic images of tumor tissue are used to predict whether a tumor is Malignant (212 instances) or Benign (357 instances). The problem is thus a binary classification problem.

In [ ]:
# Load dataset
breast_cancer_data = load_breast_cancer()
predictors = breast_cancer_data['data']
labels = breast_cancer_data['target']

# Print description of the dataset
print(breast_cancer_data['DESCR'])

The data are split in train and test sets.

In [3]:
# Parameters
seed = 0

# Train - Test Split
X_train, X_test, y_train, y_test = train_test_split(predictors, 
                                                    labels, 
                                                    random_state=seed)

## 2. CART - classification and regression trees

******

Traditional decision trees are implemented by the `DecisionTreeClassifier` and `DecisionTreeRegressor` classes in the submodule `tree` of scikit learn. 

### 2.1 Decision tree induction (learning a tree)

Decision trees can be trained and used using the default Scikit Learn API by:
- (1) creating an instance of the `DecisionTreeClassifier` class, 
- (2) call the `fit` method to train the model and 
- (3) call the `predict` method to predict the labels on test data. 

NOTE: The `DecisionTreeRegressor` class implements a tree induction algorithm for regression (not shown here).

In [5]:
# Create decision tree classifier object
decision_tree_classifier = tree.DecisionTreeClassifier(random_state=seed)

# Fit the training data to the classifier
decision_tree_classifier = decision_tree_classifier.fit(X_train, y_train)

# Calculate accuracy of the train and test sets
train_predictions = decision_tree_classifier.predict(X_train)
test_predictions = decision_tree_classifier.predict(X_test)
print("Train set accuracy is: {} and test set accuracy is: {}".format(round(accuracy_score(y_train, train_predictions), 4),
                                                                      round(accuracy_score(y_test, test_predictions), 4)))

Train set accuracy is: 1.0 and test set accuracy is: 0.8811


### 2.2 Cost-complexity pruning

Cost-complexity pruning allows you to gradually prune a tree using the cost-complexity criterion. The hyper-parameter $\alpha$ (in sklearn `cpp_alpha`) in the cost-complexity pruning acts as a regularization parameter. Larger values will result in simpler (shallow) trees.

The code fragment below sweeps over a range of values for $\alpha$ to investigate the effect of $\alpha$ on a test set. Interestingly, due to the discrete nature of decision trees, the optimal tree according to the cost-complexity criterion will depend on $\alpha$ in a discrete manner. The critical $\alpha$-values at which the cost-complexity-wise optimal tree will change can be pre-computed using the method `cost_complexity_pruning_path` of the tree instance. 

In [ ]:
# Call built-in method to compute the pruning path during Minimal Cost-Complexity Pruning.
path = decision_tree_classifier.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

# For every complexity parameter computed (cpp_alpha), fit a decision tree 

## Declare empty lists to store values
train_accuracy_list = []
test_accuracy_list = []
decision_tree_classifier_list = []

## Loop for every cost complexity pruning parameter
for idx, ccp_alpha in enumerate(ccp_alphas):
    ## Declare decision tree with cost complexity parameter
    decision_tree_classifier = tree.DecisionTreeClassifier(random_state=seed, 
                                                           ccp_alpha=ccp_alpha)
    
    ## Fit the decision tree
    decision_tree_classifier.fit(X_train, y_train)

    ## Calculate results and save the fitted decision tree
    train_predictions = decision_tree_classifier.predict(X_train)
    test_predictions = decision_tree_classifier.predict(X_test)
    train_accuracy_list.append(accuracy_score(y_train, train_predictions))
    test_accuracy_list.append(accuracy_score(y_test, test_predictions))
    decision_tree_classifier_list.append(decision_tree_classifier)

    ## Verbose 
    str_text = "Number of nodes in the tree number {}: \t {} \t-> ccp_alpha: {} \t-> train_accuracy: {} \t-> test_accuracy: {}"
    print(str_text.format(idx+1, 
                          decision_tree_classifier.tree_.node_count, 
                          str(round(ccp_alpha, 4)).ljust(6, '0'),
                          str(round(accuracy_score(y_train, train_predictions), 4)).ljust(6, '0'),
                          str(round(accuracy_score(y_test, test_predictions), 4)).ljust(6, '0')))

The effect of $\alpha$ on the train/test accuracy can be computed and visualized as shown below. Note the typical shape of the accuracy on the test-set that often occurs during hyper-parameter tuning.

In [ ]:
# Declare figure and set parameters
fig, ax = plt.subplots(figsize=(15,8))
ax.set_xlabel("cpp_alpha")
ax.set_ylabel("accuracy")
ax.set_title("Accuracy vs alpha for training and testing sets")

# Plot training data (excluding last cpp_alpha due to very low scores)
ax.plot(ccp_alphas[:-1], train_accuracy_list[:-1], marker="o", label="train", drawstyle="steps-post")

# Plot test data (excluding last cpp_alpha due to very low scores)
ax.plot(ccp_alphas[:-1], test_accuracy_list[:-1], marker="o", label="test", drawstyle="steps-post")

# Display the plot
ax.legend()
plt.show()

The code framgent below selects the best value for $\alpha$ and fits a final decision tree.

NOTE that tuning and testing rely on the same dataset here (for simplicty/illustration purposes). Ideally, the optimal value for $\alpha$ is obtained using cross validation on the training set to avoid data leakage. 

In [8]:
# Create decision tree classifier object
decision_tree_classifier = tree.DecisionTreeClassifier(random_state=seed, ccp_alpha=ccp_alphas[8])

# Fit the training data to the classifier
decision_tree_classifier = decision_tree_classifier.fit(X_train, y_train)

# Calculate accuracy of the train and test sets
train_predictions = decision_tree_classifier.predict(X_train)
test_predictions = decision_tree_classifier.predict(X_test)
print("Train set accuracy is: {} and test set accuracy is: {}".format(round(accuracy_score(y_train, train_predictions), 4),
                                                                      round(accuracy_score(y_test, test_predictions), 4)))

Train set accuracy is: 0.9648 and test set accuracy is: 0.9371


Finally, the optimal tree can be visualized as shown below.

In [ ]:
# Plot the fitted decision tree using GraphViz library
dot_data = tree.export_graphviz(decision_tree_classifier, 
                                feature_names=breast_cancer_data.feature_names,  
                                class_names=breast_cancer_data.target_names,  
                                filled=True, 
                                rounded=True,  
                                special_characters=True)  

# Display the tree plot
display(graphviz.Source(dot_data))

## 3. Learning ensembles

******

An ensemble can most easily be interpreted as a group (a set, a sequence) of models that are combined to obtain a prediction. Most ensembles rely on bagging, boosting or stacking techniques (or combinations thereof). 

### 3.1 Random Forests

Random forests is a simple, yet powerful ensemble method that relies on the construction of multiple decision trees on bootstrap replicates of a training set. The `ensemble` submodule of sklearn contains the classes `RandomForestClassifier` and `RandomForestRegressor` that implement random forests. The most important hyper-parameters are:
 * `n_estimators`: number of trees in the forest
 * `max_features`: the number of features to randomly select during *subspace projection* (the falue `'sqrt'` sets this parameter equal to the (rounded) square root of the total number of features)
 * `max_depth`: maximal depth of a tree
 * `min_samples_split`: the minimal number of instances in a node required to allow it to split further

 The code sample below fits a Random Forest classifier using defaults for the hyper-parameters

In [ ]:
# Create random forest classifier object
random_forest_classifier = ensemble.RandomForestClassifier(warm_start=True,
                                                           oob_score=True,
                                                           max_features='sqrt',
                                                           random_state=seed,)

# Fit the training data to the classifier
random_forest_classifier = random_forest_classifier.fit(X_train, y_train)

# Calculate accuracy of the train and test sets
train_predictions = random_forest_classifier.predict(X_train)
test_predictions = random_forest_classifier.predict(X_test)
print("Train set accuracy is: {} and test set accuracy is: {}".format(round(accuracy_score(y_train, train_predictions), 6),
                                                                      round(accuracy_score(y_test, test_predictions), 6)))

### 3.2 Illustration: effect of n_estimators and max_features on the accuracy

The code fragment below illustrates the effect of the number of n_estimators and max_features on the accuracy of the resulting Random Forest classifier.

In [ ]:
# Declare function to return random forest classifier with given parameters
def rf_classifier(max_features, seed):
  return ensemble.RandomForestClassifier(warm_start=True,
                                         oob_score=True,
                                         max_features=max_features,
                                         random_state=seed,)

# Declare figure and set labels
fig, ax = plt.subplots(figsize=(15,8))
ax.set_xlabel("n_estimators")
ax.set_ylabel("OOB error rate")

# Loop parameter space of max estimators and number of trees to get out of bag error scores
for max_estimator in [1, 5, 10, 30]:
  oob_score_list= []
  random_forest_classifier = rf_classifier(max_estimator, 123)

  for n_estimators in range(15, 501, 10):
    random_forest_classifier.set_params(n_estimators=n_estimators)
    random_forest_classifier = random_forest_classifier.fit(predictors, labels)
    oob_error = 1 - random_forest_classifier.oob_score_
    oob_score_list.append(oob_error)
  ax.plot(list(range(15, 501, 10)), oob_score_list, label=str(max_estimator))

# Display the plot
ax.legend(loc="upper right")
plt.show()

### 3.2 Hyperparameter tuning for Random Forests

The class `GridSearchCV` implements grid search for all models available in sklearn. It requires:
 * `param_grid`: A parameter grid to search over encoded as a dictionary where the *keys* are the names of the hyperparameters to tune and the *values* are lists that contain the values each hyperparameters should take.
 * `estimator`: the model to tune
 * `cv`: the number of folds (or a more specific cross-validation strategy, see docs)

In [16]:
# Define parameter space to search
param_grid = { 
    'max_features': [3, 10, 30],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5, 10],}

# Create random forest classifier object
random_forest_classifier = ensemble.RandomForestClassifier(n_estimators=300,
                                                           oob_score=True, 
                                                           n_jobs=-1,
                                                           random_state=seed,) 

The code fragment below performs the grid search by calling the `fit` method of a `GridSearchCV` object. Note that the `GridSearchCV` object behaves as a sklearn model. After calling `fit`, the `GridSearchCV` object behaves as a model that is trained using the optimal values for the hyper-parameters on the training set. 

In [17]:
# Perform grid search in the defined parameter space with cross validation (3 fold) -> in total 2*3*3*3 = 54 model fits 
CV_random_forest_classifier = GridSearchCV(estimator=random_forest_classifier,
                                           param_grid=param_grid,
                                           cv= 5)

CV_random_forest_classifier.fit(X_train, y_train)

# print best parameters
print('Best Parameters:', CV_random_forest_classifier.best_params_)

# Calculate accuracy of the best random forest classifier found by grid search
train_predictions = CV_random_forest_classifier.predict(X_train)
test_predictions = CV_random_forest_classifier.predict(X_test)
print("Train set accuracy is: {} and test set accuracy is: {}".format(round(accuracy_score(y_train, train_predictions), 6),
                                                                      round(accuracy_score(y_test, test_predictions), 6)))

Best Parameters: {'max_depth': 10, 'max_features': 1, 'min_samples_split': 5}
Train set accuracy is: 0.995305 and test set accuracy is: 0.937063


Alternatively, a final model can be built explicitly using the code below.

In [25]:
# Fit random forest model with best parameters
random_forest_classifier = ensemble.RandomForestClassifier(n_estimators=300,
                                                           oob_score=True, 
                                                           random_state=seed,
                                                           **CV_random_forest_classifier.best_params_,) 
random_forest_classifier = random_forest_classifier.fit(X_train, y_train)

### 3.3 Feature importances

Feature importance is an umbrella term for a set of methods that compute how much a particular features contributes to a trained model. It thus quantifies how important a feature is for the model. The code fragment below illustrates how *permutation importance* can be computed.

In [26]:
# Perform permutation feature importance using the best random forest model
permutation_importance_result = permutation_importance(random_forest_classifier, 
                                                       X_test, 
                                                       y_test, 
                                                       n_repeats=10, 
                                                       random_state=seed)

The resulting object contains the importances and their estimated standard deviations, often visualized using a bar chart.

In [ ]:
# Extract the mean and standard deviation of the feature importances from the importance object and wrap the in a Pandas Dataframe
forest_importances = pd.DataFrame({"importances" : -permutation_importance_result.importances_mean, 
                                   "stdev" : permutation_importance_result.importances_std }, 
                                   index=breast_cancer_data['feature_names']).sort_values("importances", ascending=False).iloc[:8]

# Plot the feature importances
fig, ax = plt.subplots(figsize=(15,8))
forest_importances["importances"].plot.bar(yerr=forest_importances.stdev, ax=ax)
ax.set_title("Feature importances using permutation on test data")
ax.set_ylabel("Mean accuracy decrease")
ax.set_ylim(bottom=0)
plt.show()

### 3.4 Individual Partial Dependence (ICE) plots

The code fragments below computes the individual partial dependencies and visualizes them in a partial dependence plot.

In [ ]:
# Create Pandas DataFrame for the test predictors
X_test_df = pd.DataFrame(X_test, columns=breast_cancer_data['feature_names'])

# Plot individual partial dependency of selected features 
fig, ax = plt.subplots(figsize=(20, 8))
PartialDependenceDisplay.from_estimator(random_forest_classifier, 
                                        X_test_df, 
                                        features = ['worst radius', 'worst perimeter', 'mean concave points'], 
                                        kind='both', 
                                        ax=ax)

### **Gradient Boosting Classifier with Basic Parameters**

In [ ]:
# Create a XGBoost classifier object
gradient_boosting_classifier = ensemble.GradientBoostingClassifier(n_estimators=300, 
                                                                   learning_rate=0.1,
                                                                   max_depth=3, 
                                                                   subsample=1,
                                                                   min_samples_split=2,
                                                                   min_samples_leaf=1,
                                                                   max_features='sqrt',
                                                                   random_state=seed,)

# Fit the training data to the classifier
gradient_boosting_classifier = gradient_boosting_classifier.fit(X_train, y_train)

# Calculate accuracy of the train and test sets
train_predictions = gradient_boosting_classifier.predict(X_train)
test_predictions = gradient_boosting_classifier.predict(X_test)
print("Train set accuracy is: {} and test set accuracy is: {}".format(round(accuracy_score(y_train, train_predictions), 6),
                                                                      round(accuracy_score(y_test, test_predictions), 6)))

Train set accuracy is: 1.0 and test set accuracy is: 0.958042


### 3.5 XGBoost with default parameters

XGBoost is implemented in the `xgboost` package (not part of sklearn) but the API of `xgboost` closely follows that of sklearn.

The code fragment below trains a gradiet boosting classifier with default hyper-parameters.

In [ ]:
# Create a XGBoost classifier object
xgboost_classifier = xgboost.XGBClassifier(random_state=seed,)

# Fit the training data to the classifier
xgboost_classifier = xgboost_classifier.fit(X_train, y_train)

# Calculate accuracy of the train and test sets
train_predictions = xgboost_classifier.predict(X_train)
test_predictions = xgboost_classifier.predict(X_test)
print("Train set accuracy is: {} and test set accuracy is: {}".format(round(accuracy_score(y_train, train_predictions), 6),
                                                                      round(accuracy_score(y_test, test_predictions), 6)))

XGBoost also has several hyper-parameters, the code snippet below illustrates how they can be tuned using `GridSearchCV`.

In [ ]:
# Define parameter space to search
param_grid = {
    'max_depth': [2, 3, 6],
    'learning_rate': [0.01, 0.05, 0.1, 0.3],
    'min_child_weight': [1, 10, 100],
    'colsample_bytree': [0.3, 0.7, 1],
}

# Create XGBoost classifier object
xgboost_classifier = xgboost.XGBClassifier(random_state=seed,)

# Perform grid search in the defined parameter space with cross validation (3 fold) -> in total 3*4*3*3*3 = 324 model fits 
CV_xgboost_classifier = GridSearchCV(estimator=xgboost_classifier, param_grid=param_grid, cv= 5)
CV_xgboost_classifier.fit(X_train, y_train)
print('Best Parameters:', CV_xgboost_classifier.best_params_)

# Calculate accuracy of the best XGBoost classifier found by grid search
train_predictions = CV_xgboost_classifier.predict(X_train)
test_predictions = CV_xgboost_classifier.predict(X_test)
print("Train set accuracy is: {} and test set accuracy is: {}".format(round(accuracy_score(y_train, train_predictions), 6),
                                                                      round(accuracy_score(y_test, test_predictions), 6)))

## 4. Isolation forests

***********

Isolation forests are an anomaly/outlier detection technique that relies on trees.

The code fragment below shows how Isolation Forests can be used to detect outliers in the Wisonsin Breast cancer datase and visualizes the result using t-SNE.

In [ ]:
# Create Pandas DataFrame from the predictor array
predictor_df = pd.DataFrame(predictors, columns=breast_cancer_data['feature_names'])

# Use T-distributed Stochastic Neighbor Embedding (TSNE) for creating low ddimensional embeddings (visualization purposes)
predictor_embedded = TSNE(n_components=2, 
                          learning_rate=200,
                          init='random', 
                          perplexity=10,
                          random_state=seed,).fit_transform(predictor_df)
predictor_embedded_df = pd.DataFrame(predictor_embedded, columns=['Component 1', 'Component 2'])

# Fit and predict isolation forest model
isolation_forest = ensemble.IsolationForest(n_estimators=1000, bootstrap=False, random_state=seed,).fit(predictor_df)
outliers = isolation_forest.predict(predictor_df)
outlier_scores = isolation_forest.score_samples(predictor_df)*-1

# Assign the outlier labels as a column to the Pandas DataFrame
predictor_embedded_df['outlier'] = np.where(outliers == -1, 1, np.where(outliers == 1, 0, outliers))
predictor_embedded_df['outlier_scores'] = outlier_scores

# Verbose the outlying cases indices and plot using 2 variables (outlier are marked as -1 and normal as 1)
print('Outlier Indices: \n', np.where(outliers == -1), '\n')
print('Outlier Scores: \n', predictor_embedded_df.head(n=10), '\n')
fig, ax = plt.subplots(figsize=(8, 5), dpi=100)
sns.scatterplot(x='Component 1', 
                y='Component 2', 
                hue='outlier', 
                data=predictor_embedded_df)

## 5. Auto-ML

Auto-ML is an umbrella term for techniques that automatically select and train one or several ML models without any human interaction. Auto-sklearn is an implementation of Auto-ML that scans through the library of models implemented in Sklearn and uses hyper-parameter tuning, bagging and stacking to combine these models into a large ensemble. The main input that is provided by the user is the amount of compute time the training phase is allowed to use.

In [ ]:
!pip install auto-sklearn

In [ ]:
# Import Auto-Sklearn (Run Twice!)
import autosklearn.classification

The package `autosklearn` exposes an API that mimicks that of sklearn.

In [ ]:
# Define AutoML classification model from Auto-Sklearn
automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=180)

# Fit AutoML classification model 
automl.fit(X_train, y_train)

# Calculate accuracy of the train and test sets
train_predictions = automl.predict(X_train)
test_predictions = automl.predict(X_test)
print("Train set accuracy is: {} and test set accuracy is: {}".format(round(accuracy_score(y_train, train_predictions), 6),
                                                                      round(accuracy_score(y_test, test_predictions), 6)))

Train set accuracy is: 0.995305 and test set accuracy is: 0.993007


In [ ]:
# Verbose Final Model Leaderboard from AutoML
print(automl.leaderboard())